In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from scipy.optimize import linear_sum_assignment
from model import UnifiedDetector
from criterion import Criterion
from Hier_trainer import Hier_trainer



In [ ]:
model=UnifiedDetector(im_size=128,n_classes=1).cuda()
input=torch.rand([8,3,128,128]).cuda() #[B,3,H,W]
pixel_features, mask_out, classes, semantic_mask, affinity_matrix=model(input)
print(pixel_features.shape,mask_out.shape,classes.shape,semantic_mask.shape,affinity_matrix.shape)

In [ ]:
criter=Criterion()
classes_gt=(torch.rand([8,20])+1).long().cuda()
mask_gt=F.one_hot(torch.rand([8,20,128,128]).argmax(dim=1),num_classes=20).permute(0,3,1,2).cuda()
affinity_matrix_gt=torch.rand(8,20,20).cuda()
gt_size=torch.LongTensor([6,9,2,3,4,5,5,8]).cuda()
loss=criter(classes.sigmoid().squeeze(dim=-1),classes_gt,F.softmax(mask_out,dim=1),mask_gt,affinity_matrix,affinity_matrix_gt,semantic_mask.sigmoid(),mask_gt[:,0],pixel_features,gt_size)
print(loss)

In [ ]:
trainer=Hier_trainer()
dataloader=[
    {
        'images':torch.rand([8,3,128,128]),
        'mask_gt':F.one_hot(torch.rand([8,20,128,128]).argmax(dim=1),num_classes=20).permute(0,3,1,2),
        'classes_gt': (torch.rand([8,20])+0.5).long(),
        'affinity_matrix_gt':torch.rand(8,20,20),
        'gt_size': (torch.rand(8)*10).long(),

    },
    {
        'images':torch.rand([6,3,128,128]),
        'mask_gt':F.one_hot(torch.rand([6,20,128,128]).argmax(dim=1),num_classes=20).permute(0,3,1,2),
        'classes_gt': (torch.rand([6,20])+0.5).long(),
        'affinity_matrix_gt':(torch.rand(6,20,20)+0.5).long(),
        'gt_size': (torch.rand(6)*10).long(),
    }
]
trainer.train_epoch(dataloader,0)

In [ ]:
torch.autograd.set_detect_anomaly(True)
model=UnifiedDetector(im_size=128,n_classes=1).cuda()
input=torch.rand([8,3,128,128]).cuda() #[B,3,H,W]
pixel_features, mask_out, classes, semantic_mask, affinity_matrix=model(input)
loss=mask_out.sum()
loss.backward()

In [ ]:
!python dataset.py